In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

# Define Helper Functions

In [6]:
def test_adfuller(array, array_name, alpha=0.05):
    """
    adfuller test to test for stationarity
    null hypothesis: a unit root (root >= 1 or <= -1) is present (not stationary)
    alternative hypothesis: a unit root is not present (stationary)


    Args:
        array (np.array): array to test
        array_name (string): array name to display
        alpha (float, optional): test alpha. Defaults to 0.05.
    """
    from statsmodels.tsa.stattools import adfuller

    adf_result = adfuller(array)
    adf_stats = adf_result[0]
    p_value = adf_result[1]

    print(f"{array_name}")
    print(f"\tADF Statistic: {adf_stats: .4}")
    print(f"\tp-value: {p_value: .4}")
    if p_value < alpha:
        print("\tp value < 0.05: we reject the H0 that our series is not stationary")
        print(f"\tconclusion: {array_name} is stationary")
    else:
        print(
            "\tp value > 0.05: we cannot reject the H0 that our series is not stationary"
        )
        print(f"\tconclusion: {array_name} is not stationary")


def search_optimal_diff(series, low=0, high=1):
    """find out how many times df need to be differenced to become stationary"""
    for i in range(low, (high + 1)):
        data = np.diff(series, n=i)
        test_adfuller(data, f"JJ - {i} order differencing")


def optimize_sarima(
    endog: pd.Series, ps: list, qs: list, d: int, Ps: list, Qs: list, D: int, s: int
):
    """try all possible combinatios of the hyperparameter
    credit to Marco Peixeiro

    Args:
        endog (pd.Series)
        ps (list): list of p to try
        qs (list): list of q to try
        d (int): d
        Ps (list): list of P to try
        Qs (list): list of Q to try
        D (int): D
        s (int): s

    Returns:
        pd.DataFrame: dataframe with all combinations and their AIC
    """
    import warnings
    from itertools import product
    from statsmodels.tsa.statespace.sarimax import SARIMAX
    from tqdm import tqdm

    order_list = list(
        product(ps, [d], qs, Ps, [D], Qs, [s])
    )  # get all possible combinations of (p, d, q, P, D, Q, s)

    results = []
    with warnings.catch_warnings():
        warnings.filterwarnings(
            "ignore"
        )  # ! to ignore "Using zeros as starting parameters" warning
        for order in tqdm(order_list):
            model = SARIMAX(
                endog=endog,
                simple_differencing=False,
                order=(order[0], order[1], order[2]),
                seasonal_order=(order[3], order[4], order[5], order[6]),
            )
            result = model.fit(disp=False)
            aic = result.aic
            results.append((order, aic))

    df_results = pd.DataFrame(
        results, columns=["(p, d, q, P, D, Q, s)", "aic"]
    ).sort_values("aic", ascending=True)
    return df_results


def test_ljungbox(residuals):
    """H0: The data are independently distributed (not correlated).
        Ha: The data are not independently distributed; they exhibit serial correlation.

    Args:
        residuals (residuals): SARIMAX result residuals

    Returns:
        pd.DataFrame: dataframe containing ljung-box test result
    """
    from statsmodels.stats.diagnostic import acorr_ljungbox

    df_res = acorr_ljungbox(residuals, np.arange(1, 11, 1))
    df_res = df_res.assign(
        result=np.where(
            df_res["lb_pvalue"] > 0.05,
            "residuals are not correlated",
            "residuals are correlated",
        )
    )
    return df_res

# Data Loading, Cleaning, EDA

In [3]:
train_path = "dataset/delhi-climate-data/DailyDelhiClimateTrain.csv"
test_path = "dataset/delhi-climate-data/DailyDelhiClimateTest.csv"
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)

def clean_df(df):
    return (df
        .loc[:, ["date", "meantemp"]]
        .sort_values("date", ascending=True)
        .assign(
            date=lambda df_: pd.to_datetime(df_["date"]), 
            meantemp=lambda df_: df_["meantemp"].astype("float32")
        )
        .set_index("date")
        .resample("1d")
        .ffill()
        .reset_index()
    )

df_train = clean_df(df_train)
df_test = clean_df(df_test)

print(df_train.shape, df_test.shape)
df_train.head()

(1462, 2) (114, 2)


,date,meantemp
0,2013-01-01,10.000000
1,2013-01-02,7.400000
2,2013-01-03,7.166667
3,2013-01-04,8.666667
4,2013-01-05,6.000000


# Create Dataset

In [5]:
x_train = df_train["meantemp"].values[:-114]
x_val = df_train["meantemp"].values[-114:]
x_test = df_test["meantemp"].values

x_train_looped = []
for i in range(x_train.shape[0]):
    x = np.array(x_train[i : i + 13])
    if x.shape[0] == 13:
        x_train_looped.append(x.reshape(1, 13))
x_train_looped = np.array(x_train_looped).reshape(-1, 13)


x_val_looped = []
for i in range(x_val.shape[0]):
    x = np.array(x_val[i : i + 13])
    if x.shape[0] == 13:
        x_val_looped.append(x.reshape(1, 13))
x_val_looped = np.array(x_val_looped).reshape(-1, 13)


x_test_looped = []
for i in range(x_test.shape[0]):
    x = np.array(x_test[i : i + 13])
    if x.shape[0] == 13:
        x_test_looped.append(x.reshape(1, 13))
x_test_looped = np.array(x_test_looped).reshape(-1, 13)

print(x_train_looped.shape, x_val_looped.shape, x_test_looped.shape)

(1336, 13) (102, 13) (102, 13)


# Model Selection

In [11]:
search_optimal_diff(df_train["meantemp"], low= 7, high= 7) # type: ignore

JJ - 7 order differencing
	ADF Statistic: -27.88
	p-value:  0.0
	p value < 0.05: we reject the H0 that our series is not stationary
	conclusion: JJ - 7 order differencing is stationary


In [12]:
ps = list(range(0, 4, 1))
qs = list(range(0, 4, 1))
d = 1

Ps = list(range(0, 4, 1))
Qs = list(range(0, 4, 1))
D = 1
s = 7  # number of period in a season (12 months in a year)

df_result = optimize_sarima(df_train["meantemp"], ps, qs, d, Ps, Qs, D, s)
df_result

  6%|▌         | 15/256 [00:57<19:10,  4.77s/it]